### PWW to NC


#### decoder table

In [1]:
name_decoder = {
    101: 'temp_c_2m',
    102: 'temp_F_2m',
    103: 'dewpoint_c_2m',
    104: 'dewpoint_F_2m',
    105: 'wind_speed_10m_mps',
    106: 'wind_speed_10m_mph',
    107: 'wind_direction_10m_deg',
    109: 'wind_speed_100m_mps',
    110: 'wind_speed_100m_mph',
    111: 'wind_speed_80m_to_100m_mps',
    112: 'wind_speed_80m_to_100m_mph',
    119: 'total_cloud_cover_percent',
    120: 'global_horizontal_irradiance_wm2',
    121: 'direct_horizontal_irradiance_wm2',
    122: 'vertically_integrated_smoke_mgm2',
    135: 'wind_gust_surface_mps',
    136: 'wind_gust_surface_mph',
    150: 'percent_frozen_precip_surface',
    151: 'precipitation_rate_surface_mmhr',
    1101: 'temp_c_2m_mult_100',
    1103: 'dewpoint_c_2m_mult_100',
    1105: 'wind_speed_10m_mps_mult_100',
    1109: 'wind_speed_100m_mps_mult_100',
    1120: 'global_horizontal_irradiance_wm2_full',
    1121: 'direct_horizontal_irradiance_wm2_full',
    1122: 'vertically_integrated_smoke_mgm2_full'
}

equation_decoder = {
    101: lambda x: x - 100,
    102: lambda x: x - 115,
    103: lambda x: x - 100,
    104: lambda x: x - 115,
    105: lambda x: x,  # Already in m/s
    106: lambda x: x,  # Already in mph
    107: lambda x: x * 5,  # Convert to degrees
    109: lambda x: x,  # Already in m/s
    110: lambda x: x,  # Already in mph
    111: lambda x: x,  # Conversion logic handled elsewhere
    112: lambda x: x,  # Conversion logic handled elsewhere
    119: lambda x: x,  # Already in percentage
    120: lambda x: x * 5,  # Convert to full W/m^2
    121: lambda x: x * 5,  # Convert to full W/m^2
    122: lambda x: 10 ** (x / 40),  # Convert from log to mg/m^2
    135: lambda x: x,  # Already in m/s
    136: lambda x: x,  # Already in mph
    150: lambda x: x,  # Already in percentage
    151: lambda x: x,  # Already in mm/hour
    1101: lambda x: x / 100,  # Convert to degrees C
    1103: lambda x: x / 100,  # Convert to dewpoint in C
    1105: lambda x: x / 100,  # Convert to wind speed in m/s
    1109: lambda x: x / 100,  # Convert to wind speed in m/s
    1120: lambda x: x,  # Already in full W/m^2
    1121: lambda x: x,  # Already in full W/m^2
    1122: lambda x: x  # Already in full mg/m^2
}


#### converter function

In [2]:
import struct
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from datetime import datetime, timezone, timedelta
import xarray as xr
import h5netcdf


def PWW_to_NC(pww_filename, offset_time=False):

    def read_null_terminated_string(file):
        chars = []
        while True:
            char = file.read(1)
            if char == b"\x00":
                break
            chars.append(char)
        return b"".join(chars).decode("ascii")

    with open(pww_filename, "rb") as file:

        # Read and unpack the file header
        header = struct.unpack("<hhhddddddh", file.read(56))
        start_datetime = datetime.fromtimestamp(header[3] * 86400, timezone.utc) - relativedelta(years=70, day=1)
        end_datetime = datetime.fromtimestamp(header[4] * 86400, timezone.utc) - relativedelta(years=70, day=1)

        print(f"filekey :{header[0]} {header[1]}| version:{header[2]}")
        print(f"Date range:{start_datetime} --{end_datetime}")
        print(f"lat:[{header[5]}:{header[6]}] | lon:[{header[7]}:{header[8]}]")
        if header[9]:  # if metadata is present
            print(f"file_name: {read_null_terminated_string(file)}")
        header = struct.unpack("<iiihh", file.read(16))
        unique_dates = header[0]
        LOC = header[2]
        sample_time = header[1]
        vars = header[4]
        print(f"unique_dates:{unique_dates} |LOC:{LOC} |sameple time {sample_time}s |weather_var:{header[4]}")
        vars_name = [struct.unpack("<h", file.read(2))[0] for _ in range(vars)]
        print(f"vars name:{vars_name}")
        decode_name = [name_decoder[var] for var in vars_name]
        print(f"decode name:{decode_name}")
        vars_varification = struct.unpack("<h", file.read(2))[0]
        if vars_varification != vars:
            print("vars_varification failed")
        # Read unique dates
        time_offset = timedelta(days=0) # if offset_time is True, then add 1 day to the date
        if offset_time:
            time_offset = timedelta(days=1)
        if sample_time == 0:  # if  time is present
            dates = np.array([struct.unpack("<d", file.read(8))[0] for _ in range(unique_dates)]).astype("float64")
            dates = pd.to_datetime(dates * (10**9 * 86400) - 2209161600 * 10**9, unit="ns")
            print(f"read unique dates:{dates[0]}...")
        else:
            print(f"smaple time is  present, create unique dates base on time sample {unique_dates}")
            dates = pd.date_range(start=start_datetime, end=end_datetime - time_offset, periods=unique_dates)
            # print(f"dates:{dates}")
        stations = []
        for row in range(LOC):
            lat = struct.unpack("<d", file.read(8))[0]
            lon = struct.unpack("<d", file.read(8))[0]
            alt = struct.unpack("<h", file.read(2))[0]
            # whoami = file.read(15).split(b'\x00', 1)[0].decode('ascii')
            whoami = read_null_terminated_string(file)
            country = read_null_terminated_string(file)
            region = read_null_terminated_string(file)
            stations.append((lat, lon, alt, whoami, country, region))
            # print(f"lon:{lon}, lat:{lat}, alt:{alt}, whoami:{whoami}, country:{country}, region:{region}")
        stations_df = pd.DataFrame(stations, columns=["Latitude", "Longitude", "ElevationMeters", "WhoAmI", "Country2", "Region"])
        weather_data = []
        data = np.fromfile(file, dtype=np.uint8)  #! this code didn't consider the variable type other than int8

    # base on the raw data, decode and create nc file
    lons = stations_df.Longitude.to_numpy()
    lats = stations_df.Latitude.to_numpy()
    lon_dim = np.sum(np.abs(np.diff(lons)) > 10) + 1  # the the shape of lon and lat dimension
    # print(f"lon_dim:{lon_dim}")
    data = data.reshape((unique_dates, vars, lon_dim, -1))
    print(f"data shape: times:{unique_dates}, vars:{vars}, lons:{lon_dim}, lats:{data.shape[3]}")
    data = np.where(data == 255, np.nan, data)
    time_offset = timedelta(days=0)

    ds = xr.Dataset(
        data_vars={name_decoder[var]: (["time", "lat", "lon"], equation_decoder[var](data[:, i, :, :])) for i, var in enumerate(vars_name)},
        coords={
            "time": dates,
            "latitude": (["lat", "lon"], lats.reshape(lon_dim, -1)),
            "longitude": (["lat", "lon"], lons.reshape(lon_dim, -1)),
            "region": (["lat", "lon"], stations_df.Region.to_numpy().reshape(lon_dim, -1)),
        },
    )
    ds = ds.astype(np.int16)
    # ds=ds.astype(np.float16)
    return ds

#### glob all files , convert and save

In [3]:
import glob
from tqdm import tqdm
import os

files= glob.glob(r"E:\Research-Data\Projects\weather_data\ERA5_pww\*.pww")
files= glob.glob(r"E:\Research-Data\Projects\weather_data\NOVA_FC\*.pww")
print(f"founds {len(files)} files")
files.sort()


founds 0 files


In [ ]:
ds = PWW_to_NC(file)
pd.to_datetime(ds.time.values)

In [ ]:
error=[]
dss=[]
#files=[r'E:\Research-Data\Projects\weather_data\ERA5_pww\NorthAmerica2020_Q3.pww']
for f in tqdm(files[-96:]):
    try:
        ds = PWW_to_NC(f)
        lat_min, lat_max = 25, 37  # Adjust these values as needed
        lon_min, lon_max = -107, -93  # Adjust these values as needed

        # Select the data within the specified range using the coordinates
        subset = ds.where((ds.latitude >= lat_min) & (ds.latitude <= lat_max) &
                        (ds.longitude >= lon_min) & (ds.longitude <= lon_max), drop=True)
 
        #print(ds)
        #df = ds.to_dataframe()#* in case you want to save as csv
        #ds.to_netcdf(rf'Data/weather/'+os.path.basename(f).split(".")[0]+".nc",engine='h5netcdf')
        #print(ds.lat)
        subset.to_netcdf(rf"E://Research-Data//Projects//weather_data//ERA5_TX_NC//"+os.path.basename(f).split(".")[0]+".nc",engine='h5netcdf')
    
    except Exception as e:
        print(f"Error: {e}")
        print(f"Error in file: {f}")
        error.append(f)

## test case
```

In [ ]:
file=rf"D:\Users\Thomas-chen\Documents\git\Weather-Data\weather_auto\NOAA_forecast_weather_auto\Data\pww\Forecast_NorthAmerica_Run2025-01-24T12Z.pww"
ds = PWW_to_NC(file)
ds

In [5]:
file=rf"C:\class\code\service_testing\pww_to_nc\pww\daily\20250109.pww"
ds = PWW_to_NC(file)
ds


filekey :2001 8065| version:1
Date range:2025-01-01 00:00:00+00:00 --2025-01-01 23:00:00+00:00
lat:[18.0:23.0] | lon:[-161.0:-154.0]
unique_dates:24 |LOC:609 |sameple time 3600s |weather_var:9
vars name:[102, 104, 106, 107, 119, 110, 120, 121, 136]
decode name:['temp_F_2m', 'dewpoint_F_2m', 'wind_speed_10m_mph', 'wind_direction_10m_deg', 'total_cloud_cover_percent', 'wind_speed_100m_mph', 'global_horizontal_irradiance_wm2', 'direct_horizontal_irradiance_wm2', 'wind_gust_surface_mph']
smaple time is  present, create unique dates base on time sample 24
data shape: times:24, vars:9, lons:1, lats:609


<xarray.Dataset> Size: 278kB
Dimensions:                           (time: 24, lat: 1, lon: 609)
Coordinates:
  * time                              (time) datetime64[ns, UTC] 192B 2025-01...
    latitude                          (lat, lon) float64 5kB 18.0 18.0 ... 23.0
    longitude                         (lat, lon) float64 5kB -161.0 ... -154.0
    region                            (lat, lon) object 5kB '' '' '' ... '' ''
Dimensions without coordinates: lat, lon
Data variables:
    temp_F_2m                         (time, lat, lon) int16 29kB 76 76 ... 73
    dewpoint_F_2m                     (time, lat, lon) int16 29kB 65 65 ... 69
    wind_speed_10m_mph                (time, lat, lon) int16 29kB 5 6 ... 12 13
    wind_direction_10m_deg            (time, lat, lon) int16 29kB 130 ... 110
    total_cloud_cover_percent         (time, lat, lon) int16 29kB 2 1 ... 99 98
    wind_speed_100m_mph               (time, lat, lon) int16 29kB 5 7 ... 14 14
    global_horizontal_irradiance_wm2  (time, lat, lon) int16 29kB 785 ... 315
    direct_horizontal_irradiance_wm2  (time, lat, lon) int16 29kB 695 690 ... 50
    wind_gust_surface_mph             (time, lat, lon) int16 29kB 11 12 ... 18

In [6]:
import struct
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from datetime import datetime, timezone, timedelta

#station_filename = "stationsss.parquet"
#pww_filename = r"D:\Users\Thomas-chen\Documents\git\Weather-Data\weather_auto\HRRR_weather_auto\beta_v4_sfc_01_TX.pww"
# pww_filename = r"D:\GitHub\weather_new\Weather-Data\forcast_weather\Data\pww\Forecast_NorthAmerica_Run2024-06-15T12Z"
pww_filename = file#files[-1]
print(pww_filename)


def read_null_terminated_string(file):
    chars = []
    while True:
        char = file.read(1)
        if char == b"\x00":
            break
        chars.append(char)
    #print(chars)
    return b"".join(chars).decode("ascii")
    


with open(pww_filename, "rb") as file:
   
    # Read and unpack the file header
    header = struct.unpack("<hhhddddddh", file.read(56))
    start_datetime = datetime.fromtimestamp(header[3] * 86400, timezone.utc) - relativedelta(years=70, day=1)
    end_datetime = datetime.fromtimestamp(header[4] * 86400, timezone.utc) - relativedelta(years=70, day=1)

    print(f"filekey :{header[0]} {header[1]}| version:{header[2]}")
    print(f"Date range:{start_datetime} --{end_datetime}")
    print(f"lat:[{header[5]}:{header[6]}] | lon:[{header[7]}:{header[8]}]")
    if header[9]:  # if metadata is present
        print(f"file_name: {read_null_terminated_string(file)}")
    header = struct.unpack("<iiihh", file.read(16))
    unique_dates = header[0]
    LOC = header[2]
    sample_time = header[1]
    vars = header[4]
    print(f"unique_dates:{unique_dates} |LOC:{LOC} |sameple time {sample_time}s |weather_var:{header[4]}")
    vars_name = [struct.unpack("<h", file.read(2))[0] for _ in range(vars)]
    print(f"vars name:{vars_name}")
    decode_name=[name_decoder[var] for var in vars_name]
    print(f"decode name:{decode_name}")
    vars_varification = struct.unpack("<h", file.read(2))[0]
    if vars_varification != vars:
        print("vars_varification failed")
    if sample_time == 0:  # if  time is present
        dates = np.array([struct.unpack("<d", file.read(8))[0] for _ in range(unique_dates)]).astype('float64')
        dates=pd.to_datetime(dates*(10**9 * 86400)-2209161600 * 10**9,unit='ns')
        print(f"read unique dates:{dates[0]}...")
    else:
        print(f"smaple time is present, create unique dates base on time sample {unique_dates}")
        dates = pd.date_range(start=start_datetime, end=end_datetime-timedelta(days=1), periods=unique_dates,)
        #print(f"dates:{dates}")
    stations = []
    for row in range(LOC):
        lat = struct.unpack("<d", file.read(8))[0]
        lon = struct.unpack("<d", file.read(8))[0]
        alt = struct.unpack("<h", file.read(2))[0]
        # whoami = file.read(15).split(b'\x00', 1)[0].decode('ascii')
        whoami = read_null_terminated_string(file)
        country = read_null_terminated_string(file)
        region = read_null_terminated_string(file)
        stations.append((lat, lon, alt, whoami, country, region))
        # print(f"lon:{lon}, lat:{lat}, alt:{alt}, whoami:{whoami}, country:{country}, region:{region}")
    stations_df = pd.DataFrame(stations, columns=["Latitude", "Longitude", "ElevationMeters", "WhoAmI", "Country2", "Region"])
    weather_data = []
    data = np.fromfile(file, dtype=np.uint8)#! this code didn't consider the variable type other than int8 
    #return data, dates, unique_dates
    # --------- to nc file


C:\class\code\service_testing\pww_to_nc\pww\daily\20250109.pww
filekey :2001 8065| version:1
Date range:2025-01-01 00:00:00+00:00 --2025-01-01 23:00:00+00:00
lat:[18.0:23.0] | lon:[-161.0:-154.0]
unique_dates:24 |LOC:609 |sameple time 3600s |weather_var:9
vars name:[102, 104, 106, 107, 119, 110, 120, 121, 136]
decode name:['temp_F_2m', 'dewpoint_F_2m', 'wind_speed_10m_mph', 'wind_direction_10m_deg', 'total_cloud_cover_percent', 'wind_speed_100m_mph', 'global_horizontal_irradiance_wm2', 'direct_horizontal_irradiance_wm2', 'wind_gust_surface_mph']
smaple time is present, create unique dates base on time sample 24


In [7]:
stations_df

,Latitude,Longitude,ElevationMeters,WhoAmI,Country2,Region
0,18.0,-161.00,0,+18.00-161.00/,,
1,18.0,-160.75,0,+18.00-160.75/,,
2,18.0,-160.50,0,+18.00-160.50/,,
3,18.0,-160.25,0,+18.00-160.25/,,
4,18.0,-160.00,0,+18.00-160.00/,,
...,...,...,...,...,...,...
604,23.0,-155.00,0,+23.00-155.00/,,
605,23.0,-154.75,0,+23.00-154.75/,,
606,23.0,-154.50,0,+23.00-154.50/,,
607,23.0,-154.25,0,+23.00-154.25/,,


In [8]:
from datetime import timedelta
pd.date_range(start=start_datetime, end=end_datetime, periods=745)

DatetimeIndex([          '2025-01-01 00:00:00+00:00',
               '2025-01-01 00:01:51.290322580+00:00',
               '2025-01-01 00:03:42.580645161+00:00',
               '2025-01-01 00:05:33.870967741+00:00',
               '2025-01-01 00:07:25.161290322+00:00',
               '2025-01-01 00:09:16.451612903+00:00',
               '2025-01-01 00:11:07.741935483+00:00',
               '2025-01-01 00:12:59.032258064+00:00',
               '2025-01-01 00:14:50.322580645+00:00',
               '2025-01-01 00:16:41.612903225+00:00',
               ...
               '2025-01-01 22:43:18.387096774+00:00',
               '2025-01-01 22:45:09.677419354+00:00',
               '2025-01-01 22:47:00.967741935+00:00',
               '2025-01-01 22:48:52.258064516+00:00',
               '2025-01-01 22:50:43.548387096+00:00',
               '2025-01-01 22:52:34.838709677+00:00',
               '2025-01-01 22:54:26.129032258+00:00',
               '2025-01-01 22:56:17.419354838+00:00',
         

In [9]:
import xarray as xr

lon = stations_df.Longitude.to_numpy()
lat = stations_df.Latitude.to_numpy()
lon_dim = np.sum(np.abs(np.diff(lon)) > 10) + 1  # the the shape of lon and lat dimension
data = data.reshape((unique_dates, vars, lon_dim, -1))
data=np.where(data==255, np.nan, data)

ds = xr.Dataset(
    data_vars={
        name_decoder[var]: (["time", "lat", "lon"],
        equation_decoder[var](data[:, i, :, :])) for i, var in enumerate(vars_name)},
    coords={
    "time":pd.date_range(start=start_datetime, end=end_datetime, periods=unique_dates),
    "latitude": (["lat", "lon"], lat.reshape(lon_dim, -1)),
    "longitude": (["lat", "lon"], lon.reshape(lon_dim, -1)),
    'region':(['lat','lon'],stations_df.Region.to_numpy().reshape(lon_dim, -1)),
    },
)
# plt.plot(np.diff(long))

In [10]:
test=ds.where(ds.region=="Texas", drop=True)

In [11]:
import xarray as xr

ds=xr.load_dataset(rf"D:\Users\Thomas-chen\Downloads\HRRR_TempExample.grib2")

ValueError: did not find a match in any of xarray's currently installed IO backends ['h5netcdf']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [ ]:
ds.to_dataframe().to_csv(rf"D:\Users\Thomas-chen\Downloads\HRRR_TempExample.csv")